# Advent of Code 2022 - Day 17

In [1]:
import numpy as np
from collections import defaultdict

In [2]:
test_input = """>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>"""

In [3]:
inp = open('input.txt','r').read().replace('\n',"")

In [4]:
#define coords system with respect to bottom left edge [row, column]
h_line = [[0,0], [0,1] , [0,2] , [0,3]]

cross = [ [0,0], [-1,0], [-2,0], [-1,-1], [-1,1]  ]

bend = [ [0,0], [0,1], [0,2], [-1,2], [-2,2] ]

v_line = [ [0,0] , [-1,0] , [-2,0], [-3,0]  ] 

square = [ [0,0], [0,1], [-1,0], [-1,1] ]

In [5]:
shape_coords = dict()
shape_coords['h_line'] = h_line
shape_coords['cross'] = cross
shape_coords['bend'] = bend
shape_coords['v_line'] = v_line
shape_coords['square'] = square

In [6]:
initial_point = [0,1]
[ [x[0]+initial_point[0], x[1]+initial_point[1]] for x in shape_coords['h_line'] ]

[[0, 1], [0, 2], [0, 3], [0, 4]]

In [7]:
####

class shape:
    """
    Defines coordinate systems for all our different shapes
    
    Also defines some methods of calculating coords under an attempted move left, right or down
    
    Assumes some blocked set exists which has the blocked off spaces.
    
    Also accounts for walls at 0 and 8 in the horizontal direction
    """

    def __init__(self, initial_point, geo):
        
        self.coords = [[ x[0]+initial_point[0], x[1]+initial_point[1]] for x in shape_coords[geo] ]
            
        self.left_coords = [ [x[0],x[1]-1] for x in self.coords]
        
        self.right_coords = [ [x[0],x[1]+1] for x in self.coords]
        
        self.down_coords = [ [x[0]+1,x[1]] for x in self.coords ]
        
        if geo=='cross':
            self.highest = self.coords[2][0]
        else:
            self.highest = self.coords[-1][0]
            
    def move_lateral(self, direction, blocked):
        
        if direction == '<':
            
            # my coord system doesn't quite work for the cross shape :)
            if geo == 'cross':
                if (sum([tuple(x) in blocked for x in self.left_coords ]) == 0) and (self.left_coords[3][1] != 0):
                #new initial point
                    return self.left_coords
                else:
                    return self.coords
            elif (sum([tuple(x) in blocked for x in self.left_coords ]) == 0) and (self.left_coords[0][1]!=0):
                #new initial point
                return self.left_coords
            else:
                return self.coords
        
        if direction == '>':
            if geo == 'cross':
                if (sum([tuple(x) in blocked for x in self.right_coords ]) == 0) and (self.right_coords[4][1]!=8):
                    #new initial point
                    return self.right_coords   
                else:
                    return self.coords
            
            elif (sum([tuple(x) in blocked for x in self.right_coords ]) == 0) and (self.right_coords[-1][1]!=8):
                #new initial point
                return self.right_coords
            
            else:
                return self.coords
            
        
    
    def move_down(self,blocked):
        if (sum([tuple(x) in blocked for x in self.down_coords ]) != 0) or (self.down_coords[0][0] >= 0):
            #print('hit bottom!')
            return self.coords, False
        else:
            return self.down_coords, True
    
  

In [8]:
highest_rock = 0
counter = 0
n = 0
blocked = set()
highest_rocks_seq =[]
#for part 1 we can brute force it
# limit = 2022

# a decent amount to try to identify a sequence for part 2
limit = 100000

while counter != limit:
    for geo in ['h_line','cross','bend','v_line','square']:
        #print(geo)
        if geo == 'cross':
            x=1
        else:
            x=0
        start_position = [highest_rock-4, 3+x]
        
        #initialise shape object
        current_shape = shape(start_position, geo)
        #print("initialising",current_shape.coords)
        
        motion = True
        
        while motion == True:
            
            #move it laterally
            #print("moved laterally: ", test_input[n])
            current_shape = shape(current_shape.move_lateral( inp[n] , blocked )[0],geo)
            #print(current_shape.coords)
            
            #move it down
            #print("moved down")
            new_coords,motion = current_shape.move_down(blocked)
            current_shape = shape(new_coords[0],geo)

            #print(current_shape.coords)
            
            
            if n==(len(inp)-1):
                n = 0
            else:
                n +=1
        
        #shape has been placed 
        #print("shape placed")
        counter +=1
        #print(counter)
        
        old_highest_rock = highest_rock
        
        highest_rock = min(highest_rock,current_shape.highest)
        #print("highest_rock: ",highest_rock)
        
        highest_rocks_seq.append([counter,geo, inp[n-1],abs(highest_rock),abs(highest_rock-old_highest_rock)])
        
        for x in new_coords:
            blocked.add(tuple(x))
                    
        if counter == limit:
            break
print(highest_rock)

-157594


### Part 2
Too big to loop so lets look for a cycle in our `heights added`

In [9]:
# Bit messy with dataframes here. Unnecessary with retrospect but too lazy to tidy up now.
import pandas as pd
df = pd.DataFrame(highest_rocks_seq)

#we actually only need this bit - the net height added of each rock
heights_added = df[4]

In [10]:
# function which searches through a list, expanding it's range until it finds two consecutive identical lists - e.g. a sequence

def cycle(list):
    # list to store shortest cycles
    shortest = [] 
    # return single integer and non-repeating lists
    if len(list) <= 1: return list
    if len(set(list)) == len(list): return list
    # loop through the list expanding and comparing 
    # groups of elements until a sequence is seen 
    for x in range(len(list)):
        if list[0:x] == list[x:2*x]:
            shortest = list[0:x] 
    return shortest 

In [11]:
# Key here is reversing the list as it starts with a unique list before settling into a cycle
cycle_len = len(cycle(heights_added.tolist()[::-1]))

In [12]:
#find where the sequence starts - and save this value. could defo be tidied up but <shrug>

for x in range(len(heights_added)):
    if heights_added[x:(x+cycle_len)].reset_index().drop('index',axis=1).equals(heights_added[x+cycle_len:x+(cycle_len*2)].reset_index().drop('index',axis=1)):
                print('starts repeating at rock number: ',x)
                cycle_start = x
                break

starts repeating at rock number:  93


#### Now we just need to calculate the heights added for the cycles and the bit at the start. And some fraction of a total cycle

In [13]:
# bit at the start before the cycles start
pre_cycle_value = heights_added[0:cycle_start].sum()
print(pre_cycle_value)

152


In [14]:
# height added per cycle
cycle_value = heights_added[cycle_start:cycle_start+cycle_len].sum()
cycle_value

77459

In [15]:
# rocks left once you exclude the starting bit
rocks_left = 1000000000000 - (cycle_start)
rocks_left

999999999907

In [16]:
# How many whole cycles do we have?
import math
whole_cycles = math.floor(rocks_left / cycle_len)
whole_cycles

20343810

In [17]:
# How many leftover rocks (e.g. fraction of a total cycle)
leftover = rocks_left % cycle_len
print(leftover)
leftover_value = heights_added[cycle_start:cycle_start+leftover].sum()
leftover_value

19357


30545

In [18]:
#p2 answer - putting it all together
pre_cycle_value + (whole_cycles * cycle_value) + leftover_value



1575811209487

In [19]:
#That took a while to figure out. But was sort of satisfying in the end. Am I having fun?